Import necessary libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

Load dataframe

In [3]:
df = pd.read_csv("./data/bus.csv", sep = ';', index_col = 0)
#df

Define objectives

In [4]:
### Method 1 - by a list
objectives = ['max', 'max', 'min', 'max', 'min', 'min', 'min', 'max']

### Method 2 - by a word
objectives = 'max'

### Method 3 - by a dictionary
objectives = {
    "MaxSpeed" : 'max',
    "ComprPressure" : "max",
    "Blacking" : "min",
    "Torque" : "max",
    "SummerCons" : "min",
    "WinterCons" : "min",
    "OilCons" : "min",
    "HorsePower" : "max"}

### By default all objectives are set to 'max'. Synonyms to 'max': "gain", 'g'. Synonyms to 'min': 'cost', 'c'.

Define weights

In [5]:
weights = None

Define your own range of values, by default minimal and maximal values are taken for each objective

In [6]:
expert_range = None

In [7]:
from src import MSDTransformer as msdt

Create calculator object that stores an aggregation function

Create buses object

In [8]:
# buses = msdt.MSDTransformer("A")
# buses = msdt.MSDTransformer("I")
# buses = msdt.MSDTransformer("R")
# buses = msdt.MSDTransformer(msdt.ATOPSIS)
# buses = msdt.MSDTransformer(msdt.ITOPSIS)
buses = msdt.MSDTransformer(msdt.RTOPSIS)

Before performing transformation to ranking, fit the data

In [9]:
buses.fit(df, weights, objectives,  expert_range)

Transform data to ranking

In [10]:
buses.transform(None)
print(buses.ranked_alternatives)

['b24', 'b26', 'b07', 'b16', 'b18', 'b25', 'b04', 'b01', 'b28', 'b09', 'b02', 'b13', 'b11', 'b32', 'b21', 'b12', 'b27', 'b17', 'b06', 'b29', 'b20', 'b14', 'b23', 'b19', 'b03', 'b08', 'b30', 'b15', 'b22', 'b10', 'b31', 'b05']


Check how you should change a mean of objectives for one alternative, to improve its position in the ranking

In [11]:
alternative_to_improve = buses.data.loc["b02"].copy()
alternative_to_improve_CS = buses.original_data.loc["b02"].copy()
alternative_to_overcome = buses.data.loc["b09"].copy()
improvement_ratio = 0.01
weights_mean = np.mean(buses.weights)

buses.agg_fn.improvement_mean(alternative_to_improve, alternative_to_overcome, improvement_ratio, weights_mean)

You should change mean by  0.027887731738685972


Check how you should change a sd of objectives for one alternative, to improve its position in the ranking

In [12]:
alternative_to_improve = buses.data.loc[buses.ranked_alternatives[27]].copy()
alternative_to_overcome = buses.data.loc[buses.ranked_alternatives[22]].copy()
improvement_ratio = 0.01
weights_mean = np.mean(buses.weights)

buses.agg_fn.improvement_std(alternative_to_improve, alternative_to_overcome, improvement_ratio, weights_mean)

It is impossible to improve with only standard deviation


Check how you should change particular values of features for one alternative, to improve its position in the ranking

In [13]:
alternative_to_improve = buses.data.loc[buses.ranked_alternatives[27]].copy()
alternative_to_overcome = buses.data.loc[buses.ranked_alternatives[22]].copy()
improvement_ratio = 0.01
objectives_to_change = ['MaxSpeed', 'Blacking', 'SummerCons']

buses.agg_fn.improvement_features(alternative_to_improve, alternative_to_overcome, improvement_ratio, buses.weights, objectives_to_change, buses.value_range, buses.objectives)


MaxSpeed         22.00
ComprPressure     0.00
Blacking        -44.00
Torque            0.00
SummerCons       -0.25
WinterCons        0.00
OilCons           0.00
HorsePower        0.00
Name: b15, dtype: float64


Check how you should change value of a single feature for one alternative, to improve its position in the ranking

In [14]:
alternative_to_improve = buses.data.loc["b02"].copy()
alternative_to_overcome = buses.data.loc["b09"].copy()
improvement_ratio = 1e-8
feature_to_change = 'HorsePower'

buses.agg_fn.improvement_features(alternative_to_improve, alternative_to_overcome, improvement_ratio, buses.weights, [feature_to_change],
                                  buses.value_range, buses.objectives)

lower_bounds = buses.original_data.min().values.copy()
upper_bounds = buses.original_data.max().values.copy()
alternative_to_improve_CS = buses.original_data.loc["b02"].copy()
buses.agg_fn.improvement_single_feature(alternative_to_improve, alternative_to_overcome, improvement_ratio, buses.weights, feature_to_change, buses.value_range, buses.objectives, alternative_to_improve_CS, lower_bounds, upper_bounds)

MaxSpeed         0.000000
ComprPressure    0.000000
Blacking         0.000000
Torque           0.000000
SummerCons       0.000000
WinterCons       0.000000
OilCons          0.000000
HorsePower       6.332868
Name: b02, dtype: float64
Solutions: 6.332870829555235 39.333299024606205
Criterion HorsePower is gain type, we need to increase value
Only solution_1 is feasible
HorsePower needs to be improved by 6.332870829555235


Check transformation of "new" alternatives

In [15]:
result1 = buses.data.iloc[:, -3:]
result2 = pd.DataFrame(buses.transform_new_data(df.values, normalize_data=True), index=["Mean", "Std", "AggFn"], columns=df.index).T
np.allclose(result1, result2)

True